In [246]:
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import gmaps
from scipy.stats import linregress
from pprint import pprint
from geopy.geocoders import Nominatim
from datetime import datetime,date
from uszipcode import SearchEngine
zip_search = SearchEngine(simple_zipcode=False) # set simple_zipcode=False to use rich info database

# Google developer API key
from api_keys import g_key
from api_keys import p_key

# Configure gmaps
gmaps.configure(api_key=g_key)


In [121]:
marvel_df = pd.read_json("Cleaned_Marvel_Dating_AmericanJSON.json")
#marvel_df

In [122]:
marvel_df2 = marvel_df[['CODENAME','BIRTHPLACE']]
#marvel_df2

In [123]:
#Now we're going to get the zip codes, latitudes, and longitudes.

#define variables
indices = []
drop_indices = []


#Grab indicies for loopin'
indices = list(marvel_df2.index.values)

# initialize Nominatim API (geopy.geocoders)
geolocator = Nominatim(user_agent="geoapiExercises")


In [251]:
#Loop it for zip codes
zip_list = []
lat_list = []
lon_list = []
city_list = []
state_list = []
codename_list = []

results_dict = {"CODENAME":[],"City":[],"State":[],"Zip Code":[],"Latitude":[],"Longitude":[]}
n = 0
test_indices = np.arange(0,10)

for index in indices:
    try:
        #place is the inputs for geolocator, namely address, city, state by index number
        place = f"{marvel_df2['BIRTHPLACE'][index]}"
        
        #Grab location data results from geolocator
        location = geolocator.geocode(place)
        post_code = geolocator.reverse((location.latitude, location.longitude))
        
        #Grab City and State from 
        #loc2 = zip_search.by_zipcode(results_dict['Zip Code'][index])
        
        #append results to the to the dictionary
        results_dict['Zip Code'].append(post_code.raw['address']['postcode'])
        results_dict['Latitude'].append(location.latitude)
        results_dict['Longitude'].append(location.longitude)
        #results_dict['City'].append(loc2.major_city)
        #results_dict['State'].append(loc2.state)
        results_dict['CODENAME'].append(marvel_df2['CODENAME'][index])
        
        #codename = results_dict["CODENAME"][n]
        #print(f"TRY1 {index}: {codename_list[index]} City: {city_list[index]} State: {state_list[index]} Zip Code: {zip_list[index]} Latitude: {lat_list[index]} Longitude: {lon_list[index]}")
        #n = n+1
        
    except:
        results_dict['Zip Code'].append("NULL")
        results_dict['Latitude'].append("NULL")
        results_dict['Longitude'].append("NULL")
        #results_dict['City'].append("NULL")
        #results_dict['State'].append("NULL")
        results_dict['CODENAME'].append(marvel_df2['CODENAME'][index])
        drop_indices.append(index)
        #print(f"EXP {index}: {codename_list[index]} City: {city_list[index]} State: {state_list[index]} Zip Code: {zip_list[index]} Latitude: {lat_list[index]} Longitude: {lon_list[index]}")

for index in indices:
    try:
        loc2 = zip_search.by_zipcode(results_dict['Zip Code'][index])
        results_dict['City'].append(loc2.major_city)
        results_dict['State'].append(loc2.state)
        #print(f"TRY: {index} {results_dict['CODENAME'][index]}, {results_dict['City'][index]}, {results_dict['State'][index]}")
    except:
        results_dict['City'].append("NULL")
        results_dict['State'].append("NULL")
        #print(f"EXP: {index} {results_dict['CODENAME'][index]}, {results_dict['City'][index]}, {results_dict['State'][index]}")
 
print(len(results_dict['CODENAME']))
print(len(results_dict['Latitude']))
print(len(results_dict['Longitude']))
print(len(results_dict['Zip Code']))
print(len(results_dict['City']))
print(len(results_dict['State']))
#pprint(results_dict)

results_df = pd.DataFrame(results_dict)
results_df

1272
1272
1272
1272
1272
1272


,CODENAME,City,State,Zip Code,Latitude,Longitude
0,Spider-Man,Flushing,NY,11358,40.7498,-73.7976
1,Captain America,New York,NY,10155,40.7588,-73.9623
2,The Thing,None,None,10000,40.7127,-74.006
3,Mister Fantastic,Clarksburg,CA,95612,38.3927,-121.573
4,Human Torch,Schenectady,NY,12302,42.8683,-73.9283
...,...,...,...,...,...,...
1267,Black Talon,New Orleans,LA,70130,29.9499,-90.0701
1268,Tumbler,Newark,NJ,07102,40.7357,-74.1724
1269,Commuter,Newark,NJ,07102,40.7357,-74.1724
1270,Star-Thief,None,None,NULL,NULL,NULL


In [177]:
test_indices = np.arange(0,10)

for index in test_indices:
    try:
        #Grab location data results from geolocator
        post_code = geolocator.reverse((results_dict['Latitude'][index], results_dict['Longitude'][index]))
        state_list.append(post_code.raw['address']['state'])
        city_list.append(post_code.raw['address']['city'])
        print(f"TRY1 {index} {results_dict['CODENAME'][index]} City: {city_list[index]} State: {state_list[index]}")

    except:
        try:
            state_list.append(post_code.raw['address']['state'])
            city_list.append(post_code.raw['address']['municipality'])
            print(f"TRY2 {index} {results_dict['CODENAME'][index]} City: {city_list[index]} State: {state_list[index]}")

        except:
            try:
                state_list.append(post_code.raw['address']['state'])
                city_list.append(post_code.raw['address']['county'])
                print(f"TRY3 {index} {results_dict['CODENAME'][index]} City: {city_list[index]} State: {state_list[index]}")

            except:
                state_list.append("NULL")
                city_list.append("NULL")
                print(f"EXP {index} {results_dict['CODENAME'][index]} City: {city_list[index]} State: {state_list[index]}")


TRY1 0 Spider-Man City: NULL State: NULL
TRY1 1 Captain America City: NULL State: NULL
TRY1 2 The Thing City: NULL State: NULL
TRY3 3 Mister Fantastic City: Bulgurlu Mahallesi State: NULL
TRY2 4 Human Torch City: NULL State: NULL
TRY2 5 Invisible Woman City: NULL State: NULL
TRY3 6 Hawkeye City: NULL State: NULL
TRY1 7 Daredevil City: NULL State: NULL
TRY1 8 Doctor Strange City: NULL State: NULL
TRY3 9 Mary Jane Watson City: NULL State: NULL


In [139]:
print(codename_list)
print(lat_list)
print(lon_list)
print(zip_list)
print(city_list)
print(state_list)

['Spider-Man',
 'Captain America',
 'The Thing',
 'Mister Fantastic',
 'Human Torch',
 'Invisible Woman',
 'Hawkeye',
 'Daredevil',
 'Doctor Strange',
 'Mary Jane Watson',
 'J. Jonah Jameson',
 'Giant-Man',
 'Nick Fury',
 'Wasp',
 'Vision',
 'May Parker',
 'Captain Marvel',
 'Hulk',
 'The Punisher',
 'Luke Cage',
 'Robbie Robertson',
 'Norman Osborn',
 'Agent Anti-Venom',
 'Wonder Man',
 'Winter Soldier',
 'Betty Brant',
 'Rick Jones',
 'Hellcat',
 'Human Torch',
 'Foggy Nelson',
 'Spider-Man',
 'War Machine',
 'Dum Dum Dugan',
 'Falcon',
 'Mayor Fisk',
 'Black Knight',
 'Kindred',
 'Powerhouse',
 'Harpy',
 'Kid Colt',
 'Red Hulk',
 'Moon Knight',
 'Gwen Stacy',
 'Nova',
 'Mockingbird',
 'Tigra',
 'Moonstone',
 'Sandman',
 'Sharon Carter',
 'Black Cat']
[40.7498243,
 40.7587979,
 40.7127281,
 38.392688,
 42.8682502,
 42.8682502,
 42.7258142,
 40.7644228,
 39.9527237,
 41.2552213,
 40.735657,
 40.7318045,
 40.7127281,
 40.9414874,
 40.7127281,
 40.735657,
 42.3602534,
 34.0536909,
 40.7

In [185]:
index = 0
post_code = geolocator.reverse((results_dict['Latitude'][index], results_dict['Longitude'][index]))
#loc2 = geolocator.geocode(post_code)
print(marvel_df2['CODENAME'][index])
#print(place)
#pprint(location.raw)
pprint(post_code.raw['address']['city'])
#print(f"{index}: {codename_list[index]} Zip Code: {zip_list[index]} Latitude: {lat_list[index]} Longitude: {lon_list[index]}")

Spider-Man
'New York'


In [109]:
print(f"{len(codename_list)} Codenames")
print(f"{len(lat_list)} Latitude")
print(f"{len(lon_list)} Longitude")
print(f"{len(zip_list)} Zip Codes")
print(f"{len(city_list)} Cities")
print(f"{len(state_list)} States")

print(np.arange(0,10))

1 Codenames
1 Latitude
1 Longitude
1 Zip Codes
0 Cities
0 States
[0 1 2 3 4 5 6 7 8 9]


In [253]:
results_df1 = results_df.drop(columns='CODENAME')
results_df1

,City,State,Zip Code,Latitude,Longitude
0,Flushing,NY,11358,40.7498,-73.7976
1,New York,NY,10155,40.7588,-73.9623
2,None,None,10000,40.7127,-74.006
3,Clarksburg,CA,95612,38.3927,-121.573
4,Schenectady,NY,12302,42.8683,-73.9283
...,...,...,...,...,...
1267,New Orleans,LA,70130,29.9499,-90.0701
1268,Newark,NJ,07102,40.7357,-74.1724
1269,Newark,NJ,07102,40.7357,-74.1724
1270,None,None,NULL,NULL,NULL


In [254]:
final_results_df = marvel_df.join(results_df1, how='left')
final_results_df

,_id,TARGET,CODENAME,REAL NAME,GENDER,RELATIONSHIP STATUS,ALIGNMENT,HEIGHT,WEIGHT,EYECOLOR,...,FIGHTING,PROFILE PIC,FAVORITE HANGOUTS,DETAILED FILE,POWERS URL,City,State,Zip Code,Latitude,Longitude
0,{'$oid': '60ba7853eb3d706d33244c7e'},Spider-Man,Spider-Man,Peter Benjamin Parker,Male,Single,Good,70,167.0,Hazel,...,4,https://static.wikia.nocookie.net/marveldataba...,"New York City, New York",https://marvel.fandom.com/wiki/Peter_Parker_(E...,https://marvel.fandom.com/wiki/Peter_Parker_(E...,Flushing,NY,11358,40.7498,-73.7976
1,{'$oid': '60ba7853eb3d706d33244c7f'},Captain America,Captain America,"Steven ""Steve"" Rogers",Male,Single,Good,74,240.0,Blue,...,6,https://static.wikia.nocookie.net/marveldataba...,"Avengers Mountain, North Pole, Arctic Circle, ...",https://marvel.fandom.com/wiki/Captain_America...,https://marvel.fandom.com/wiki/Captain_America...,New York,NY,10155,40.7588,-73.9623
2,{'$oid': '60ba7853eb3d706d33244c80'},Benjamin Grimm,The Thing,"Benjamin Jacob ""Ben"" Grimm",Male,Married,Good,72,500.0,Blue,...,5,https://static.wikia.nocookie.net/marveldataba...,"4 Yancy Street, Lower East Side, Manhattan, Ne...",https://marvel.fandom.com/wiki/Benjamin_Grimm_...,https://marvel.fandom.com/wiki/Benjamin_Grimm_...,None,None,10000,40.7127,-74.006
3,{'$oid': '60ba7853eb3d706d33244c81'},Reed Richards,Mister Fantastic,Dr. Reed Richards,Male,Married,Good,73,180.0,Brown,...,3,https://static.wikia.nocookie.net/marveldataba...,"4 Yancy Street, Manhattan, New York City, New ...",https://marvel.fandom.com/wiki/Reed_Richards_(...,https://marvel.fandom.com/wiki/Reed_Richards_(...,Clarksburg,CA,95612,38.3927,-121.573
4,{'$oid': '60ba7853eb3d706d33244c82'},Jonathan Storm,Human Torch,"Jonathan Lowell Spencer ""Johnny"" Storm",Male,Divorced,Good,70,170.0,Blue,...,3,https://static.wikia.nocookie.net/marveldataba...,"formerly Avengers Mansion, Manhattan, New York...",https://marvel.fandom.com/wiki/Jonathan_Storm_...,https://marvel.fandom.com/wiki/Jonathan_Storm_...,Schenectady,NY,12302,42.8683,-73.9283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,{'$oid': '60ba7853eb3d706d33245171'},Thibodaux Boudreaux,Black Talon,Thibodaux Boudreaux,Male,Single,Bad,74,240.0,Brown,...,2,https://static.wikia.nocookie.net/marveldataba...,"New Orleans, Louisiana",https://marvel.fandom.com/wiki/Thibodaux_Boudr...,https://marvel.fandom.com/wiki/Thibodaux_Boudr...,New Orleans,LA,70130,29.9499,-90.0701
1268,{'$oid': '60ba7853eb3d706d33245172'},Michael Keane,Tumbler,Michael Keane,Male,Single,Good,71,175.0,MultiColored,...,4,https://static.wikia.nocookie.net/marveldataba...,"Manhattan, New York City, New York",https://marvel.fandom.com/wiki/Michael_Keane_(...,https://marvel.fandom.com/wiki/Michael_Keane_(...,Newark,NJ,07102,40.7357,-74.1724
1269,{'$oid': '60ba7853eb3d706d33245173'},Commuter,Commuter,Ron,Male,Married,Bad,72,170.0,MultiColored,...,2,https://static.wikia.nocookie.net/marveldataba...,"Scarsdale, New York",https://marvel.fandom.com/wiki/Commuter_(Ron)_...,https://marvel.fandom.com/wiki/Commuter_(Ron)_...,Newark,NJ,07102,40.7357,-74.1724
1270,{'$oid': '60ba7853eb3d706d33245174'},Ilse Pterigil,Star-Thief,Ilse Pterigil,Female,Single,Neutral,56,87.0,White,...,4,https://static.wikia.nocookie.net/marveldataba...,Dreolln,https://marvel.fandom.com/wiki/Ilse_Pterigil_(...,https://marvel.fandom.com/wiki/Ilse_Pterigil_(...,None,None,NULL,NULL,NULL


In [279]:
final_2_df = final_results_df.drop(axis="index", drop_indices, inplace=False)
final_2_df

SyntaxError: positional argument follows keyword argument (<ipython-input-279-49002706988d>, line 1)

In [281]:
cleaned_csv = pd.read_csv("output/Marvel_Interim_cleaned.csv")
cleaned_df = pd.DataFrame(cleaned_csv)
cleaned_df

,Unnamed: 0,_id,TARGET,CODENAME,REAL NAME,GENDER,RELATIONSHIP STATUS,ALIGNMENT,HEIGHT,WEIGHT,...,FIGHTING,PROFILE PIC,FAVORITE HANGOUTS,DETAILED FILE,POWERS URL,City,State,Zip Code,Latitude,Longitude
0,0,{'$oid': '60ba7853eb3d706d33244c7e'},Spider-Man,Spider-Man,Peter Benjamin Parker,Male,Single,Good,70,167.0,...,4,https://static.wikia.nocookie.net/marveldataba...,"New York City, New York",https://marvel.fandom.com/wiki/Peter_Parker_(E...,https://marvel.fandom.com/wiki/Peter_Parker_(E...,Flushing,NY,11358,40.749824,-73.797634
1,1,{'$oid': '60ba7853eb3d706d33244c7f'},Captain America,Captain America,"Steven ""Steve"" Rogers",Male,Single,Good,74,240.0,...,6,https://static.wikia.nocookie.net/marveldataba...,"Avengers Mountain, North Pole, Arctic Circle, ...",https://marvel.fandom.com/wiki/Captain_America...,https://marvel.fandom.com/wiki/Captain_America...,New York,NY,10155,40.758798,-73.962343
2,3,{'$oid': '60ba7853eb3d706d33244c81'},Reed Richards,Mister Fantastic,Dr. Reed Richards,Male,Married,Good,73,180.0,...,3,https://static.wikia.nocookie.net/marveldataba...,"4 Yancy Street, Manhattan, New York City, New ...",https://marvel.fandom.com/wiki/Reed_Richards_(...,https://marvel.fandom.com/wiki/Reed_Richards_(...,Clarksburg,CA,95612,38.392688,-121.573011
3,4,{'$oid': '60ba7853eb3d706d33244c82'},Jonathan Storm,Human Torch,"Jonathan Lowell Spencer ""Johnny"" Storm",Male,Divorced,Good,70,170.0,...,3,https://static.wikia.nocookie.net/marveldataba...,"formerly Avengers Mansion, Manhattan, New York...",https://marvel.fandom.com/wiki/Jonathan_Storm_...,https://marvel.fandom.com/wiki/Jonathan_Storm_...,Schenectady,NY,12302,42.868250,-73.928279
4,5,{'$oid': '60ba7853eb3d706d33244c83'},Susan Storm,Invisible Woman,"Dr. Susan ""Sue"" Storm Richards",Female,Married,Good,66,120.0,...,3,https://static.wikia.nocookie.net/marveldataba...,"New York City, New York",https://marvel.fandom.com/wiki/Susan_Storm_(Ea...,https://marvel.fandom.com/wiki/Susan_Storm_(Ea...,Schenectady,NY,12302,42.868250,-73.928279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,1266,{'$oid': '60ba7853eb3d706d33245170'},Pinky Fusser,Fusion,Hubert & Pinky Fusser,Male,Single,Neutral,48,80.0,...,2,https://static.wikia.nocookie.net/marveldataba...,"New York City, New York",https://marvel.fandom.com/wiki/Pinky_Fusser_(E...,https://marvel.fandom.com/wiki/Pinky_Fusser_(E...,Newark,NJ,7102,40.735657,-74.172367
1028,1267,{'$oid': '60ba7853eb3d706d33245171'},Thibodaux Boudreaux,Black Talon,Thibodaux Boudreaux,Male,Single,Bad,74,240.0,...,2,https://static.wikia.nocookie.net/marveldataba...,"New Orleans, Louisiana",https://marvel.fandom.com/wiki/Thibodaux_Boudr...,https://marvel.fandom.com/wiki/Thibodaux_Boudr...,New Orleans,LA,70130,29.949932,-90.070116
1029,1268,{'$oid': '60ba7853eb3d706d33245172'},Michael Keane,Tumbler,Michael Keane,Male,Single,Good,71,175.0,...,4,https://static.wikia.nocookie.net/marveldataba...,"Manhattan, New York City, New York",https://marvel.fandom.com/wiki/Michael_Keane_(...,https://marvel.fandom.com/wiki/Michael_Keane_(...,Newark,NJ,7102,40.735657,-74.172367
1030,1269,{'$oid': '60ba7853eb3d706d33245173'},Commuter,Commuter,Ron,Male,Married,Bad,72,170.0,...,2,https://static.wikia.nocookie.net/marveldataba...,"Scarsdale, New York",https://marvel.fandom.com/wiki/Commuter_(Ron)_...,https://marvel.fandom.com/wiki/Commuter_(Ron)_...,Newark,NJ,7102,40.735657,-74.172367


In [283]:
results3_df = cleaned_df.drop(cleaned_df.columns[[0]],axis=1,inplace=False)
results3_df

,_id,TARGET,CODENAME,REAL NAME,GENDER,RELATIONSHIP STATUS,ALIGNMENT,HEIGHT,WEIGHT,EYECOLOR,...,FIGHTING,PROFILE PIC,FAVORITE HANGOUTS,DETAILED FILE,POWERS URL,City,State,Zip Code,Latitude,Longitude
0,{'$oid': '60ba7853eb3d706d33244c7e'},Spider-Man,Spider-Man,Peter Benjamin Parker,Male,Single,Good,70,167.0,Hazel,...,4,https://static.wikia.nocookie.net/marveldataba...,"New York City, New York",https://marvel.fandom.com/wiki/Peter_Parker_(E...,https://marvel.fandom.com/wiki/Peter_Parker_(E...,Flushing,NY,11358,40.749824,-73.797634
1,{'$oid': '60ba7853eb3d706d33244c7f'},Captain America,Captain America,"Steven ""Steve"" Rogers",Male,Single,Good,74,240.0,Blue,...,6,https://static.wikia.nocookie.net/marveldataba...,"Avengers Mountain, North Pole, Arctic Circle, ...",https://marvel.fandom.com/wiki/Captain_America...,https://marvel.fandom.com/wiki/Captain_America...,New York,NY,10155,40.758798,-73.962343
2,{'$oid': '60ba7853eb3d706d33244c81'},Reed Richards,Mister Fantastic,Dr. Reed Richards,Male,Married,Good,73,180.0,Brown,...,3,https://static.wikia.nocookie.net/marveldataba...,"4 Yancy Street, Manhattan, New York City, New ...",https://marvel.fandom.com/wiki/Reed_Richards_(...,https://marvel.fandom.com/wiki/Reed_Richards_(...,Clarksburg,CA,95612,38.392688,-121.573011
3,{'$oid': '60ba7853eb3d706d33244c82'},Jonathan Storm,Human Torch,"Jonathan Lowell Spencer ""Johnny"" Storm",Male,Divorced,Good,70,170.0,Blue,...,3,https://static.wikia.nocookie.net/marveldataba...,"formerly Avengers Mansion, Manhattan, New York...",https://marvel.fandom.com/wiki/Jonathan_Storm_...,https://marvel.fandom.com/wiki/Jonathan_Storm_...,Schenectady,NY,12302,42.868250,-73.928279
4,{'$oid': '60ba7853eb3d706d33244c83'},Susan Storm,Invisible Woman,"Dr. Susan ""Sue"" Storm Richards",Female,Married,Good,66,120.0,Blue,...,3,https://static.wikia.nocookie.net/marveldataba...,"New York City, New York",https://marvel.fandom.com/wiki/Susan_Storm_(Ea...,https://marvel.fandom.com/wiki/Susan_Storm_(Ea...,Schenectady,NY,12302,42.868250,-73.928279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,{'$oid': '60ba7853eb3d706d33245170'},Pinky Fusser,Fusion,Hubert & Pinky Fusser,Male,Single,Neutral,48,80.0,Grey,...,2,https://static.wikia.nocookie.net/marveldataba...,"New York City, New York",https://marvel.fandom.com/wiki/Pinky_Fusser_(E...,https://marvel.fandom.com/wiki/Pinky_Fusser_(E...,Newark,NJ,7102,40.735657,-74.172367
1028,{'$oid': '60ba7853eb3d706d33245171'},Thibodaux Boudreaux,Black Talon,Thibodaux Boudreaux,Male,Single,Bad,74,240.0,Brown,...,2,https://static.wikia.nocookie.net/marveldataba...,"New Orleans, Louisiana",https://marvel.fandom.com/wiki/Thibodaux_Boudr...,https://marvel.fandom.com/wiki/Thibodaux_Boudr...,New Orleans,LA,70130,29.949932,-90.070116
1029,{'$oid': '60ba7853eb3d706d33245172'},Michael Keane,Tumbler,Michael Keane,Male,Single,Good,71,175.0,MultiColored,...,4,https://static.wikia.nocookie.net/marveldataba...,"Manhattan, New York City, New York",https://marvel.fandom.com/wiki/Michael_Keane_(...,https://marvel.fandom.com/wiki/Michael_Keane_(...,Newark,NJ,7102,40.735657,-74.172367
1030,{'$oid': '60ba7853eb3d706d33245173'},Commuter,Commuter,Ron,Male,Married,Bad,72,170.0,MultiColored,...,2,https://static.wikia.nocookie.net/marveldataba...,"Scarsdale, New York",https://marvel.fandom.com/wiki/Commuter_(Ron)_...,https://marvel.fandom.com/wiki/Commuter_(Ron)_...,Newark,NJ,7102,40.735657,-74.172367


In [284]:
#output_data_file = "output/Clean_Marvel_Dating_American.json"
result = results3_df.to_json(r'output/Cleaned_Marvel_Dating_American_v3.json')
result = results3_df.to_csv(r'output/Cleaned_Marvel_Dating_American_v3.csv')
#parsed = json.loads(result)
#print(json.dumps(parsed, indent=4))